# 普通名詞の獲得
本ノートブックでは普通名詞の獲得について仮説を立てて検証する。

最初に検証する仮説はこちら:

* 辞書ベースでラベリングした際、正解部分以外にラベリングした場合は普通名詞である可能性が高い


まずは必要なモジュールをインポートする。

In [17]:
import os
import sys
sys.path.append('..')

from seqeval.metrics import f1_score

from entitypedia.evaluation.converter import to_iob2
from entitypedia.knowledge_ner.recognizer import KnowledgeBaseRecognizer
from entitypedia.corpora.wikipedia.extractor import load_jsonl

まずは辞書ベースの分類器を読み込む。

In [18]:
recognizer = KnowledgeBaseRecognizer()
file_path = '../data/interim/title_entity.jsonl'
for d in load_jsonl(file_path):
    entity, label = list(d.items())[0]
    sub_type = label.split('/')[-1]
    label = label.split('/')[0]
    recognizer.add_entity(entity, label)
    recognizer.add_word(entity, sub_type, page_url='', image_url='')
recognizer.build()

除外するタイプを定義する。

In [19]:
remove_types = {'volume', 'period_date', 'percent', 'url', 'service', 'multiplication',
                'n_person', 'school_age', 'seismic_intensity', 'period_month',
                'phone_number', 'rank', 'n_animal', 'countx_other', 'point',
                'periodx_other', 'calorie', 'space', 'period_time', 'n_country',
                'n_product', 'numex_other', 'latitude_longtitude', 'id_number',
                'n_flora', 'facility_part', 'temperature', 'weight', 'age', 'water_root',
                'n_natural_object_other', 'intensity', 'time', 'n_facility',
                'n_organization', 'postal_address', 'period_year', 'ordinal_number',
                'physical_extent', 'speed', 'measurement_other', 'seismic_magnitude',
                'n_event', 'period_week', 'frequency', 'ignored', 'stock', 'n_location_other'}

次に正解データを読み込む。

In [20]:
BASE_DIR = '../data/raw/corpora'
mainichi_dir = os.path.join(BASE_DIR, 'mainichi')
X_true, y_true = to_iob2(mainichi_dir, remove_types)

正解データを読み込んだら、辞書ベース分類器を用いて、ラベル付する。

In [21]:
y_pred = [recognizer.iob2(text) for text in X_true]

以上で仮説検証の準備ができた。ここからは実際に検証していく。

まずは正解ラベルが付いていない箇所にラベル付している部分を洗い出す。

In [22]:
for y_p, y_t, char in zip(y_pred[0], y_true[0], X_true[0]):
    if y_t == 'O' and y_p != 'O':
        print('{}\t{}'.format(char, y_p))

国	B-other
際	I-other
な	B-city
ど	B-company
点	B-weapon
ビ	B-company
ジ	I-company
ョ	I-company
ン	I-company
の	B-city
基	B-position_vocation
本	I-position_vocation
す	B-position_vocation
は	B-city
の	B-city
会	I-position_vocation
見	I-position_vocation
で	B-company
と	B-position_vocation
す	B-position_vocation
る	B-position_vocation
所	B-company
を	B-position_vocation
発	B-other
表	I-other
し	B-company
た	I-company
月	B-astral_body_other
中	B-weapon
に	B-city
を	B-position_vocation
む	B-city
グ	B-company_group
ル	I-company_group
ー	I-company_group
プ	I-company_group
が	B-insect
発	B-other
表	I-other
す	B-position_vocation
る	B-position_vocation
の	B-city
基	B-position_vocation
本	I-position_vocation
的	B-drug
な	B-city
え	B-position_vocation
を	B-position_vocation
し	B-company
た	I-company
も	B-city
の	B-city
わ	B-river
が	B-insect
国	B-country
に	B-city
ふ	B-position_vocation
さ	B-weapon
わ	I-weapon
し	B-weapon
い	I-weapon
国	B-other
際	I-other
に	B-city
よ	B-city
る	B-position_vocation
世	B-domestic_region
界	I-domestic_region
平	B-war
和	I-war
の	B-ci

上記を分析すると以下のことがわかる。

* 曖昧さ回避ページが多い
* 全体的には普通名詞であることが多い
* 固有名詞の場合もある(e.g. 新 -> 昔の中国の国名)

仮説としては悪くなさそうなので、さらに検証を進めていく。
上記は文字単位でついているので、単語単位に変換して分析する。

In [23]:
from seqeval.metrics.sequence_labeling import get_entities

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

In [24]:
words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
words[0:10]

['国際', 'な', 'ど', '点', 'ビジョン', 'の', '基本', 'す', 'は', 'の']

ここまでで、ラベル付結果を単語単位に変換しました。

得られた単語の統計情報を確認してみます。確認する統計情報は以下の通り。

* トークン数(延べ単語数)
* タイプ数(異なり語数)
* 各単語の頻度

In [25]:
from collections import Counter

freq = Counter(words)
freq100 = sum(v for _, v in freq.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1702430
Num of types  : 22852
1223686 / 1702430
の	113730
に	67129
を	57958
は	54075
い	52871
が	52026
る	49761
と	49416
で	48583
た	44416
て	44150
な	35477
し	28469
っ	26763
れ	24690
か	24502
ら	22181
も	21313
す	19664
だ	15968
した	15784
り	15429
こ	15266
う	15244
ま	13846
き	12148
く	11936
め	11249
さ	11151
け	10775
ど	10729
あ	10238
え	8419
そ	7845
一	7150
ん	7149
や	7124
人	6737
つ	6645
年	6593
わ	5647
時	5123
大	4880
中	4430
ち	4307
ろ	4157
ば	4132
分	4074
的	3968
み	3836
よう	3803
よ	3728
日	3638
せ	3590
行	3467
お	3294
ず	3110
じ	3020
べ	2938
へ	2875
本	2781
高	2686
間	2676
目	2607
方	2446
げ	2435
しい	2356
要	2337
はな	2322
対	2308
内	2265
生	2177
ほ	2144
場	2128
取	2079
受	2052
力	2042
強	2039
下	2032
初	1922
月	1912
国	1900
新	1866
勝	1865
通	1844
実	1836
とも	1834
地	1816
午	1792
調	1775
む	1737
合	1727
手	1712
び	1690
金	1684
半	1679
かし	1676
現在	1640
話	1627
自	1624
千	1593
数	1592
明	1574
元	1568
円	1547
全	1536
せん	1534
点	1510
問題	1447
表	1447
体	1425
面	1377
ご	1369
また	1349
交渉	1323
連	1320
向	1304
政府	1295
なかった	1266
書	1259
かな	1255
私	1255
度	1243
ぶ	1214
子	1205
より	1199
支	119

飛び込み	20
ピンチ	20
群馬	20
三塁打	20
分類	20
入力	20
マグマ	20
ロシア	20
指令	20
九九	20
生物	20
それぞれに	20
測定	20
合成	20
切り札	20
千葉県	20
刃	20
都心	20
模型	20
シン	20
速報	20
国旗	20
マイ	20
拘置	20
繁華街	20
コレクション	20
フラ	20
右翼	20
ジャンル	20
封筒	20
青学	20
逆行	20
軍隊	20
ドル	20
シングル	20
ピン	20
連帯	20
内陸	20
主義	20
叫び	20
廃棄物	20
ゴルフ	20
チェーン	20
首長	20
シア	20
慶大	20
落下	20
環境保護団体	20
航空機	20
通路	20
課長	20
医院	20
ハウス	20
合併症	20
秘密会	20
はら	19
ハウ	19
少子化	19
酔い	19
衛星	19
財布	19
不良	19
広大	19
塔	19
送金	19
廊下	19
瞬	19
趙治勲	19
下着	19
同志	19
石川	19
中山	19
泉	19
消滅	19
北東	19
前夜	19
土産	19
群れ	19
音声	19
収穫	19
開拓	19
舌	19
海水	19
ちなみ	19
電波	19
一緒	19
改革派	19
出店	19
博物館	19
戸籍	19
社会保障	19
転倒	19
ディフェンス	19
岡崎	19
大島	19
絵本	19
山中	19
泡	19
酸	19
ダブル	19
丈	19
長い間	19
スケジュール	19
全国大会	19
共和国	19
大方	19
出来高	19
歳出	19
暴落	19
伝説	19
であい	19
第三者	19
大物	19
木材	19
離脱	19
ドーム	19
聖	19
めし	19
島田	19
時間帯	19
容量	19
土砂	19
信頼性	19
盾	19
ドライバー	19
遺跡	19
解剖	19
期日	19
家賃	19
ガキ	19
ドラフト	19
峰	19
推測	19
復興計画	19
オープン戦	19
朝日	19
表記	19
利子	19
池田市	19
法律相談	19
カラ	19
配当	19
コウ	19
違法性	19
財閥	19
オフレコ	19
誠実	18
粉	18
独身	18
人道	18
パフォーマンス	18
やな	18
プレート	18
顧問	18
近代化	18


代理店	6
教育改革	6
大三	6
台湾	6
私立大学	6
地方財政	6
内蔵	6
斬	6
よいこ	6
窯	6
文体	6
暗記	6
日刊	6
経済力	6
記念写真	6
中流	6
家事労働	6
順位戦	6
足音	6
ウー	6
フィーバー	6
封じ手	6
岡工	6
愛媛	6
守山	6
徳島市	6
室蘭	6
永井	6
コー	6
飯島	6
京セラ	6
福留	6
矢野	6
篠原	6
コロンビア	6
サウジアラビア	6
気圧配置	6
大雪	6
自作	6
寿命	6
ツル	6
三台	6
リバ	6
車輪	6
無口	6
第三世代	6
ボイス	6
余韻	6
悲鳴	6
まったり	6
在野	6
雪国	6
アクシデント	6
精通	6
太刀	6
ネス	6
ブラ	6
アメリ	6
シーツ	6
夜明け	6
古都	6
ゆがみ	6
公権力	6
暦	6
初日の出	6
ドーン	6
食欲	6
カタログ	6
小口	6
小手	6
護憲	6
研究開発	6
ミサイル	6
マイカ	6
電力会社	6
ヒーター	6
なっち	6
人生観	6
宇宙開発	6
学力	6
さらし	6
アンカー	6
女体	6
タスキ	6
負け越し	6
米子	6
平岡	6
大和証券	6
調査会社	6
漫才	6
気象台	6
ゴンドラ	6
発電所	6
草分け	6
王朝	6
カラフル	6
ベイエリア	6
ツイン	6
西一	6
くらま	6
短命	6
親切	6
入植地	6
社会環境	6
ペットボトル	6
皮膚	6
火災保険	6
煮物	6
ホワイトカラー	6
先祖	6
文中	6
愛想	6
フォア	6
竜巻	6
単調	6
お返し	6
大木	6
東日本	6
松尾	6
バージョン	6
生放送	6
ローカル	6
萩	6
専務	6
日東	6
府中市	6
尿	6
ターミナル	6
特別会	6
ゆう	6
歌謡曲	6
のぶ	6
謡	6
ブロー	6
新しい世界	6
外貨準備	6
購買力	6
あのこ	6
題名	6
領主	6
高砂	6
正月	6
チャイ	6
ニヤ	6
戸田	6
茅	6
多摩市	6
チリ	6
乾杯	6
地方銀行	6
注意報	6
リハーサル	6
カバ	6
あんた	6
屯	6
五十嵐広三	6
市場規模	6
電話回線	6
関根真	6
シワ	6
南国	6
規律	6
流行語	6
ハマ	6
北九州市	6
暗転	6
起業家	6
反転	6
国際通貨	6
抑留	6
独学	6
高松市	6
ほろ	

ノミ	3
たけし	3
ビエンナーレ	3
光栄	3
探究	3
象嵌	3
かえし	3
風化	3
獅子舞	3
都市化	3
リトル	3
陣取り	3
宗主国	3
雑居ビル	3
長時間労働	3
投了	3
塔頭	3
名跡	3
岩本薫	3
木谷	3
山城宏	3
歯車	3
滅私奉公	3
過労死	3
オク	3
郡山	3
吉原	3
多々良	3
山上	3
宣誓	3
横田	3
仲村	3
九電工	3
安川電機	3
高崎	3
コニカ	3
松村	3
スズキ	3
日立電線	3
クロサキ	3
胡	3
芙蓉	3
エスビー食品	3
愛知製鋼	3
川村	3
原口	3
堀川	3
藤村	3
鎌	3
富永	3
オーストリア	3
チェコ	3
バドミントン	3
ノルウェー	3
陛下	3
空へ	3
参道	3
かすみ	3
築地	3
コネ	3
ノーチ	3
ロビー活動	3
戦中	3
防寒着	3
一路	3
帆船	3
ボラ	3
故事	3
満天の星	3
病院船	3
医師	3
さえずり	3
自転車旅行	3
まるごと	3
カーン	3
集団生活	3
大理	3
迷子	3
ホウ	3
ファー	3
ウール	3
冒険者	3
大冒険	3
瞳	3
副産物	3
後	3
ラム	3
戦士	3
ファイル	3
ヒヤヒヤ	3
原子力潜水艦	3
魅せられて	3
トト	3
レーニ	3
ジム	3
ファイト	3
ウインチ	3
ジグザグ	3
パラシュート	3
ホルダー	3
大富豪	3
新興国	3
挑め	3
スターズ	3
イル	3
モロ	3
マリン	3
船尾	3
日照	3
精神世界	3
漁村	3
欠落	3
電力量	3
森林破壊	3
観光資源	3
休眠	3
文化遺産	3
大家族	3
生活環境	3
法曹	3
工業地	3
いかだ	3
民族音楽	3
初版	3
供託	3
受信機	3
方法論	3
友愛	3
自力更生	3
遊園地	3
最高速度	3
電柱	3
電気料金	3
社会貢献	3
イズ	3
バーン	3
腕時計	3
合金	3
どうしてだろう	3
西濃運輸	3
オフサイド	3
伏線	3
ハーフタイム	3
地区対抗	3
白鴎	3
沼田	3
大山	3
大和田	3
沼津	3
二十二社	3
一条	3
フロッピーディスク	3
動画	3
要点	3
二人の女	3
川口市	3
北大塚	3
クリーニング	3
勝浦	3
河口湖	3
軽井沢	3
八丈島	3
深浦	3
鉄塔	3
製本	3
北中	3
更衣室	3


須田	2
グラブ	2
シンカー	2
日向	2
キーマン	2
スプリング	2
上野桜木	2
ウナ	2
急行電車	2
物質文明	2
教区	2
木更津市	2
金町	2
ミノ	2
入学試験	2
管理者	2
涙ながらに	2
大神	2
コラ	2
見	2
リスク回避	2
トレーダー	2
スピードスキー	2
オーキッド	2
公傷	2
東横	2
西大	2
武蔵川	2
東関	2
高田川	2
豊ノ海	2
鳴戸	2
春野	2
ある子供	2
危険信号	2
関税同盟	2
リュックサック	2
カザフ	2
海岸通	2
十字路	2
聴診	2
ゐ	2
文庫本	2
夜会	2
固有名詞	2
免税店	2
エベレスト	2
パラパラ	2
紡錘	2
新木場	2
ビデオモニター	2
和太鼓	2
廃盤	2
悲恋	2
電器店	2
ドキッ	2
健全性	2
吸収合併	2
館町	2
相原町	2
都市問題	2
公害対策	2
メモ帳	2
羽生善治	2
社主	2
富山市	2
カンボジア	2
ゴースト	2
遠方から	2
ヒューズ	2
家庭訪問	2
債務超過	2
光線	2
西部劇	2
独り言	2
外国為替市場	2
エンドレス	2
まちまち	2
がぶ飲み	2
宣戦布告	2
締切	2
フィードバック	2
工学	2
瀬戸内海	2
合意形成	2
評論家	2
弾劾	2
季節調整	2
鉱工業指数	2
中平	2
東京市場	2
貿易摩擦	2
アト	2
スコール	2
役作り	2
仮名	2
転送	2
ナイス	2
まんま	2
日没	2
ゴーゴーゼット	2
アグネス	2
がんばろう	2
強心臓	2
健太郎	2
黒石	2
水谷	2
球速	2
三村	2
秦	2
星野	2
山田和	2
璧	2
ゼンリン	2
棒高跳	2
田中健	2
利根	2
大滝	2
ニオール	2
内山	2
ロッキー・リン	2
ペリ	2
ミオ	2
ビスマルク	2
東京銀行	2
読経	2
デン	2
イナ	2
ホスピス	2
自然観	2
夷	2
平沢	2
国民休暇村	2
羽田	2
大分市	2
広路町	2
鳥栖市	2
前栽	2
中将	2
家柄	2
四股名	2
ぐるり	2
張り手	2
力学	2
ツイ	2
温泉宿	2
カセット	2
ズームイン	2
マジ	2
ガン	2
遅咲き	2
木枯らし	2
光合成	2
飼料	2
使役	2
非営利団体	2
弾き語り	2
老女	2
オアシス	2
石畳	2
迷路	2
キナ	2
メイ

経営企画	1
基本粒子	1
光速	1
ヒッグス粒子	1
巨大科学	1
基礎研究	1
基礎科学	1
ただ乗り	1
反感情	1
日本人	1
岡宏	1
ホステス	1
岡田温	1
葛城	1
スカート	1
橋本康	1
いつのまに	1
互換機	1
ワークステーション	1
海運	1
定先	1
混沌	1
ファニ	1
レーガン	1
ブッシュ政権	1
佐官	1
スローダウン	1
オーバーキル	1
ダック	1
原敬	1
ブルーライト	1
いしだあゆみ	1
舟木一夫	1
リンゴの唄	1
並木路子	1
おはじき	1
お手玉	1
すごろく	1
福笑い	1
戦災孤児	1
妻側	1
女ひとり	1
離婚届	1
日親	1
新刊	1
サキ	1
フォーメーション	1
アタッカー	1
スリー	1
西陵	1
東京アカデミー	1
常盤貴子	1
西村和彦	1
思いをはせることだけは自由	1
新川博	1
たらこ	1
同人誌	1
北中部	1
防風林	1
ブリッジ	1
皿相	1
上目黒	1
上野公園	1
上賀茂	1
南千束	1
ネオンサイン	1
吉祥寺東町	1
矢切	1
上用賀	1
上野毛	1
高取	1
過料	1
熱処理	1
電気泳動	1
装置産業	1
サイエンス	1
歴史街道	1
ぃ	1
地方気象台	1
八王寺	1
明野町	1
下河原通	1
八坂	1
第三の男	1
天野建	1
伊賀貞雪	1
内藤町	1
ゾン	1
帰省ラッシュ	1
燭	1
ゴミ袋	1
辛党	1
公衆便所	1
パーキングエリア	1
セッション	1
ももち	1
化学工業	1
会地村	1
駒場	1
花言葉	1
メルヘン	1
フィッティングルーム	1
紫	1
瑠璃	1
藤波	1
天然素材	1
すり身	1
保存料	1
太巻き	1
開運	1
あきな	1
途中下車	1
深い森	1
ナイト	1
グランド・オペラ	1
タンブリン	1
メガホン	1
ボヘミアン	1
の美	1
シネ	1
史上最大の作戦	1
テクノ	1
手締め	1
初荷	1
視程	1
マンハッタン計画	1
世界大戦	1
科学者	1
技術者	1
材木座	1
鮮魚店	1
モルタル	1
河原町	1
ノースウエスト航空	1
雪洞	1
女友達	1
ナウい	1
キカ	1
マラン	1
ブリュッセル	1
くす玉	1
協会	1
議員立法	1
旗頭	1
中央高速道路	1
行政委員会	1
戦略物資	1
ペンディング	1
経済評論家

蔵王町	1
今津町	1
松永町	1
田尻町	1
大坪町	1
八本松町	1
深安郡	1
神辺町	1
楚	1
首里	1
具志川市	1
宜野湾市	1
浦添市	1
糸満市	1
糸満	1
沖縄市	1
島尻郡	1
豊見城	1
我那覇	1
南風原町	1
長戸	1
大街道	1
宮田町	1
六軒家	1
宇和島市	1
新居浜市	1
西条市	1
大洲市	1
伊予三島市	1
伊予郡	1
上之町	1
仏生山町	1
古新町	1
西宝町	1
明神	1
田村町	1
仁井田	1
中浜	1
愛宕山	1
赤石町	1
高須	1
はりまや	1
五台山	1
ナスカ	1
布師田	1
南国市	1
後免町	1
佐賀市	1
嘉瀬	1
唐津市	1
轟	1
下関市	1
彦島	1
長府	1
侍	1
竹崎町	1
山の田	1
山口市	1
米屋町	1
徳山市	1
久米	1
貞宗	1
楠木	1
晴海	1
防府市	1
松崎町	1
原田村	1
戎	1
下松市	1
岩国市	1
光市	1
新南陽市	1
新蔵町	1
山城町	1
東浜	1
南庄町	1
助任本町	1
南末広町	1
川内町	1
加賀	1
須野	1
撫養町大桑島	1
撫養町	1
大津町徳長	1
小松島市	1
小松島町	1
阿南市	1
日開野町	1
宮原	1
橘町	1
豊浜	1
門司区	1
大里	1
若松区	1
中川町	1
戸畑区	1
小倉南区	1
八幡東区	1
白川町	1
香椎	1
月隈	1
六本松	1
城南区	1
早良区	1
有田	1
西新	1
大牟田市	1
久留米市	1
鞍	1
東和町	1
筑紫野市	1
春日市	1
大野城市	1
宗像市	1
太宰府市	1
観世音寺	1
志免町	1
志免	1
古賀町	1
嘉穂郡	1
穂波町	1
宮崎市	1
堀川町	1
熊野	1
椿山	1
赤江	1
江田	1
生野	1
峰町	1
日向市	1
田迎	1
大江	1
上通	1
下通	1
上熊本	1
八代市	1
荒尾市	1
菊池郡	1
合志町	1
上益城郡	1
益城町	1
武岡	1
大明丘	1
玉里団地	1
船津町	1
上福元町	1
下福元町	1
政所	1
都町	1
三芳	1
八多	1
原川	1
海原	1
別府市	1
高平	1
高尾町	1
秋月	1
田上町	1
中園町	1
曙町	1
土井首	1
牧島町	1
船越町	1
西彼杵郡	1
長与町	1
尻無川	1
中電	1
紙皿	1
タンクローリー	1
保育者	1


職員会議	1
おかっぱ	1
バード	1
ネイチャー	1
食品衛生	1
残留農薬	1
郷土愛	1
張り子	1
がらんどう	1
ウルトラマラソン	1
紅一点	1
残暑	1
フレームワーク	1
日運	1
傍若無人	1
若月	1
阿部一	1
田中一	1
和え物	1
ゆり	1
酒井一	1
小沼	1
太平洋安全保障条約	1
安全保障条約	1
ロン	1
ボルジャー	1
地球市民	1
社会運動	1
欧州共同体	1
欧州統合	1
マーストリヒト	1
欧州中央銀行	1
高柳先男	1
毎日出版文化賞	1
国名	1
アドリア海	1
映像記憶	1
デジタルビデオ	1
クリアランス	1
電卓	1
パズル	1
粗鋼	1
化学品	1
汎用	1
準備金	1
約定	1
ちゃぶ台	1
ソルジャー	1
ねじまき鳥クロニクル	1
鳥刺し	1
村上春樹	1
神戸新聞社	1
小室直樹	1
渡部昇一	1
スキップ	1
北村薫	1
服部真澄	1
祥伝社	1
パラサイト・イヴ	1
瀬名秀明	1
フアン	1
黒子	1
だれも知らない小さな国	1
佐藤さとる	1
小学館	1
緞帳	1
すだれ	1
大夫	1
ナーシ	1
東陽	1
グレート金山	1
ワタナベ	1
松島二郎	1
ロット	1
スズキ・カバト	1
セニ	1
頭突き	1
コミッション	1
ボクサー	1
フェイント	1
漢人	1
黒鳥文絵	1
坂田和也	1
石井めぐみ	1
金奉準	1
ストロー	1
大橋秀行	1
レオ・ガメス	1
ベネズエラ	1
田所	1
君津	1
山梨球友クラブ	1
摂津	1
光シーガルズ	1
渡辺哲	1
三重殺	1
春日井	1
岩下	1
ヒルマン	1
セーブポイント	1
アウエ	1
丸池町	1
参加賞	1
メディア研究	1
英語	1
ホームドラマ	1
朝の連続ドラマ	1
即位	1
王位	1
継続審議	1
沈降	1
常温	1
海外領土	1
郷土史	1
幕臣	1
浅間温泉	1
南野	1
三者面談	1
た会	1
浦賀水道	1
議	1
広東料理	1
しらゆき	1
土地家屋調査士会	1
乳房マッサージ	1
バリア	1
内部障害	1
津波警報	1
肝移植	1
成長障害	1
骨密度	1
免疫寛容	1
教育相談	1
適応指導教室	1
復水器	1
熱交換器	1
タービン	1
始球式	1
ディスポーザー	1
東塚	1
ながら運転	1
動物愛護週間	1
強運	1
スナッ

全体的な感想:

* 曖昧さ回避ページが多い
* 固有名詞も結構多いように感じる

固有名詞が多いのはどういうことなんだろう？

* 正解データの質が悪い?
  * 辞書ベースではついたのに正解にはついていない
* 正解データの読み込み方にバグがある?
* `to_iob2`関数でignoredを弾いたのに問題がある?

おそらく、`to_iob2`関数に原因がある。というわけで修正して再びやってみる。

In [26]:
X_true, y_true = to_iob2(mainichi_dir, {})
y_pred = [recognizer.iob2(text) for text in X_true]

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
freq = Counter(words)
freq100 = sum(v for _, v in freq.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1543995
Num of types  : 14200
1156096 / 1543995
の	108630
に	65662
を	56701
は	52035
い	51308
が	50957
る	48821
と	48303
で	46448
た	43462
て	42759
な	34610
し	27673
っ	26002
れ	24157
か	23814
ら	21535
も	20748
す	18923
だ	15706
した	15399
こ	14967
う	14882
り	14878
ま	12186
く	11548
め	10967
き	10834
け	10432
ど	10402
あ	10011
さ	9897
え	8221
そ	7668
や	6961
ん	6117
つ	5948
わ	5486
ち	4148
ろ	4033
ば	3982
的	3909
よう	3692
み	3660
よ	3605
大	3526
せ	3485
行	3330
年	3144
中	3091
ず	2973
お	2921
じ	2914
べ	2880
一	2790
へ	2777
人	2690
時	2496
げ	2333
方	2320
要	2296
しい	2295
はな	2288
対	2210
内	2077
分	2038
取	2033
ほ	2026
力	1982
受	1979
強	1957
場	1951
高	1947
目	1827
とも	1809
初	1804
実	1787
地	1700
間	1692
下	1681
通	1674
む	1669
合	1669
かし	1651
手	1618
び	1610
生	1608
新	1599
調	1583
国	1582
話	1580
日	1566
自	1509
元	1493
金	1492
全	1484
問題	1424
明	1420
本	1394
面	1335
体	1322
また	1321
政府	1282
ご	1275
向	1260
なかった	1252
かな	1215
私	1212
書	1200
ぶ	1183
化	1174
支	1165
より	1124
ね	1116
主	1106
定	1101
関係	1100
直	1093
気	1087
連	1081
反	1062
変	1047
いの	1038
今年	1034
何	1024
まれ	1012
数	100

講義	28
汚職	28
職務	28
南部	28
気配	28
四球	28
猛暑	28
ホール	28
アクション	28
偽造	28
投稿	28
引き揚げ	28
上告	28
着工	28
ケア	28
対面	28
フォーラム	28
断層	28
歯止め	27
はち	27
神話	27
進歩	27
バブル経済	27
我慢	27
ポスター	27
権威	27
印刷	27
顧客	27
今でも	27
ロマン	27
共感	27
ヨット	27
ハン	27
かま	27
典型	27
数多	27
資本	27
拠出	27
シンボル	27
栽培	27
商店街	27
庭	27
国債	27
戦術	27
フリー	27
ニーズ	27
コツ	27
討論	27
班	27
拍車	27
退院	27
人質	27
列車	27
棒	27
ロボット	27
才	27
排出	27
ヘリコプター	27
会期	27
すす	27
清算	27
市街地	27
キャリア	27
幹部	27
痴漢	27
情報公開	26
誤解	26
経由	26
倫理	26
もう一度	26
敗戦	26
皮肉	26
彫刻	26
国籍	26
私立	26
オレ	26
持続	26
交付	26
出馬表	26
参拝	26
飛行機	26
通達	26
繁殖	26
徒歩	26
疎開	26
テスト	26
農薬	26
現行犯	26
海岸	26
視聴者	26
暫	26
洗い	26
半導体	26
パソコン通信	26
あし	26
ワープ	26
国交	26
さぶ	26
工程	26
ゼネコン	26
捕虜	26
神経	26
出張	26
破産	26
ロケ	26
補助	26
ありがとう	26
ゲスト	26
消印	26
人工	26
代打	26
キャッチ	25
党大会	25
杯	25
区域	25
奥さん	25
瓶	25
大地	25
世紀	25
丘	25
森林	25
あこがれ	25
摩擦	25
動力	25
時計	25
少数民族	25
玄関	25
継承	25
オイ	25
大部	25
研究会	25
亡命	25
医薬品	25
本質	25
貯金	25
育児	25
ボイコット	25
栄養	25
偶然	25
累	25
ブロック	25
公団	25
図書館	25
工業	25
規則	25
恋	25
子会社	25
ペット	25
のん	25
敬称	25
本土	25
控訴	25
現	25
打率	25
行部	25
毒ガス	25
高齢化

宝物	6
建築物	6
ターゲット	6
路線バス	6
頂上	6
専門紙	6
国有林	6
月日	6
パンク	6
公共工事	6
総合病院	6
ディスク	6
バッシング	6
電話料金	6
石像	6
まなざし	6
デート	6
くるま	6
トレンド	6
ほら	6
初夢	6
産	6
人民	6
失速	6
階層	6
社会不安	6
気質	6
選手権	6
笛	6
花道	6
ワイドショー	6
礼拝堂	6
血圧	6
燃費	6
愛する	6
密猟	6
宮中	6
凧	6
ゲームソフト	6
クイズ	6
獣	6
パン	6
黒星	6
断面	6
始業式	6
トタン	6
東照宮	6
史料	6
巡礼	6
フェア	6
バンカー	6
ビック	6
ヌード	6
従軍	6
スクープ	6
酸性雨	6
踊り場	6
生き残れ	6
ミク	6
はさみ	6
バン	6
闘士	6
新劇	6
ボーダーレス	6
カリキュラム	6
永久	6
人体実験	6
間取り	6
トリプル	6
唇	6
スタメン	6
胴上げ	6
運動会	6
へのこ	6
ハレ	6
生きよう	6
ジョイント	6
気球	6
地名	6
野生動物	6
線引き	6
相応	6
デー	6
現況	6
天体	6
かなえ	6
有害	6
湿原	6
もず	6
帰郷	6
任官	6
未開	6
思い込み	6
書評	6
タネ	6
ジレンマ	6
造形	6
曲線	6
垂直	6
ミニチュア	6
泣きながら	6
状況判断	6
林業	6
殺処分	6
変造	6
振り出し	6
反核運動	6
外装	6
記念に	6
コーラス	6
アンコール	6
ドッキング	6
短冊	6
義務教育	6
定説	6
通信社	6
諸君	6
デイサービス	6
異議	6
探索	6
加速度	6
魔女	6
在籍	6
単行本	6
実技	6
酢	6
高速鉄道	6
政府首脳	6
くぎ	6
相性	6
最前線	6
発掘調査	6
ニュータウン	6
座り込み	6
田園	6
失職	6
とうし	6
人工島	6
被害届	6
言論の自由	6
前文	6
修繕	6
独居老人	6
在家	6
郵便番号	6
宇宙ステーション	6
石碑	6
家紋	6
潜水	6
民謡	6
仮装	6
淀	6
くず	6
青空	6
カウンセリング	6
福祉事務所	6
上機嫌	6
賃貸住宅	6
ハンス	6
断絶	6
書き下ろし	6
職人	6
保存食	6
ビデオカメラ	6
玉砕	6
体当たり	6
総務	6
瓦	6


宇宙人	3
べが	3
生理	3
不当な取引制限	3
適材適所	3
多数決	3
密約	3
旅立ち	3
スポーツカー	3
野獣	3
珠	3
取り付け騒ぎ	3
クモ	3
直角	3
一撃	3
屋上	3
横顔	3
ポア	3
通信機器	3
複写	3
スポーツクラブ	3
公開審査	3
コレクトコール	3
政府税	3
公営競技	3
写真店	3
エンパワーメント	3
公案	3
衝動	3
コロ	3
国大	3
三	3
茶髪	3
マウスピース	3
子供部屋	3
善悪	3
注射器	3
被覆	3
流し台	3
幼鳥	3
石灰岩	3
霊感商法	3
事実認定	3
特融	3
所見	3
被告	3
専門チャンネル	3
凛	3
だんな様	3
大樹	3
スローイン	3
女性週刊誌	3
架空口座	3
喫煙率	3
古紙	3
統合軍	3
紅葉	3
いとう	3
ポピュラー音楽	3
新しい歌	3
お召	3
ハッピーエンド	3
淵	3
腹話術	3
スク	3
ごみ拾い	3
自習	3
山寺	3
オートマチック	3
国家公務員	3
耐震診断	3
答弁書	3
連対	3
ダート	3
ヘルプ	3
根抵当権	3
フラノ	3
自由市場	3
公訴時効	3
エナメル	3
スティック	3
東西大学	3
オフェンス	3
カーフ	3
掃海艇	3
ともち	3
リーチ	3
放電	3
国造	2
キラリ	2
実生	2
手綱	2
計数	2
晩婚化	2
予防外交	2
ラリス	2
ボーテ	2
大きな政府	2
下野	2
真空地帯	2
スパ	2
みなみ	2
爆弾	2
傍受	2
海軍基地	2
新たな時代	2
社債	2
教育制度	2
凪	2
衆参同日選挙	2
針路	2
シラ	2
擬態	2
溢れる	2
モラ	2
頭脳流出	2
拳法	2
新雪	2
性的虐待	2
新しい波	2
太陽	2
勝負師	2
ピックアップ	2
おはよう	2
オク	2
繊	2
両輪	2
お七夜	2
お印	2
冬型	2
断筆	2
空襲警報	2
蔵元	2
年賀	2
リセ	2
戦中	2
防寒着	2
ララ	2
氷床	2
ボラ	2
砂塵	2
さじ	2
もっと強く	2
豪農	2
病院船	2
水平線	2
刺激臭	2
ハプニング	2
思春期	2
民芸品	2
カーン	2
レトルト	2
固有種	2
寄宿舎	2
島国	2
馬賊	2
名士	2
偉人	2
そうかな	2
エキゾチシズム	2
エス	2
ピオ	2
熱気球	2
魅せられ

パン焼き器	1
微分	1
世界中どこを探しても	1
グリーンコンシューマー	1
トレイ	1
割賦販売	1
下取り	1
平均賃金	1
事実審	1
クマ	1
登校拒否児	1
人間社会	1
吸入	1
一人っ子	1
テクニシャン	1
ツボ	1
グロ	1
ギー	1
命綱	1
オピニオン	1
記者の目	1
中古車	1
会議室	1
情報検索	1
ハンドルネーム	1
性道徳	1
ギャグ漫画	1
掟	1
日照時間	1
レッド	1
工事監理	1
断線	1
スタンプラリー	1
会話教室	1
トロイの木馬	1
予約販売	1
振込	1
協力会社	1
みその	1
分離派	1
対句	1
の降伏文書	1
文科	1
めざまし	1
の地理	1
いくさ	1
摩擦音	1
パンタグラフ	1
二八	1
新実	1
用地買収	1
反人種差別	1
法規	1
デジタルオーディオ	1
デジタル信号	1
パーカッション	1
ノイズ	1
ピアノ曲	1
クラクラ	1
ロア	1
サッパ	1
南の島	1
芝居茶屋	1
広小路	1
蜻蛉	1
義賊	1
蛤刃	1
しかと	1
ろう石	1
義歯	1
パビリオン	1
除草剤	1
畦	1
節電	1
調査書	1
仮面	1
林間	1
文学史	1
排水路	1
望遠レンズ	1
鍋料理	1
絹織物	1
の後	1
競輪場	1
食通	1
大好物	1
順応性	1
帝都	1
海上都市	1
ケーソン	1
顧客情報	1
消費税	1
やぶさかでない	1
プレスリリース	1
驚愕	1
トゥルース	1
フリースペース	1
こせん	1
ビルトイン	1
ドングリ	1
市場価格	1
労働争議	1
四分	1
海外派遣	1
も連	1
元老	1
郷鎮	1
海峡	1
急がば回れ	1
火消	1
遍歴	1
身請	1
傾向と対策	1
スポーツ刈り	1
大食	1
火力発電	1
フリートーク	1
しりとり	1
安否情報	1
交通情報	1
う段	1
アナグラム	1
課長	1
モスラ	1
転勤族	1
レトルト食品	1
湿布	1
観音開き	1
まりも	1
パイル	1
居間	1
シャギー	1
ホリ	1
一石二鳥	1
やる気満々	1
ミニゲーム	1
紅白	1
街頭募金	1
グローブ	1
戦争と平和	1
水商売	1
浄土	1
三昧	1
如来	1
安楽	1
大三	1
オープンカレッジ	1
カッパドキア	1
薪ストーブ	1
朝日	1
仕置	1
の空	1
雲

ブーメラン	1
レディ	1
アイディ	1
の壁崩壊	1
悲喜劇	1
妖精	1
口承文学	1
ゴスペル	1
ボーカ	1
清澄	1
ポジティブ	1
フレンドリー	1
ダーク	1
レプリカ	1
置石	1
小目	1
三々	1
ルネサンス	1
すず	1
本題	1
われら	1
貪	1
家職	1
刀剣	1
禁欲	1
代役	1
刑事法	1
アチーブメントテスト	1
タックスアンサー	1
南地	1
水遊び	1
航空機の検査	1
ガバメント	1
動物実験	1
芦屋	1
忙殺	1
耐震工事	1
ちょび	1
勘当	1
おもろ	1
コメディアン	1
ギャバジン	1
編み物	1
自打球	1
での福祉	1
有限会社	1
地方競馬	1
直接販売	1
アッツ島	1
サングラス	1
ゆく河の流れは絶えずして	1
うたかた	1
新書	1
いいじゃありませんか	1
基本法	1
岡部三郎	1
院内会派	1
ミスター	1
専守防衛	1
コンパクト化	1
山口	1
染色体異常	1
放射線療法	1
声帯	1
ユートピア	1
渡り鳥	1
で債	1
美意識	1
変数	1
異常気象	1
天気図	1
気象情報	1
寡占	1
盧	1
民主化宣言	1
軍事革命	1
抵抗勢力	1
バンパー	1
集会の自由	1
霊視	1
告訴・告発	1
公職	1
断食	1
ヒット商品番付	1
交信	1
公理	1
国民体育大会	1
ムリ	1
動物虐待	1
とまん	1
水部	1
バーター	1
議定書	1
生産関係	1
盆栽	1
ログハウス	1
万国旗	1
しんしん	1
毛無山	1
共同浴場	1
外湯	1
不意打ち	1
茎	1
金策	1
コザ	1
釘	1
・ムーブメント	1
ドリームチーム	1
長野	1
サック	1
女子プロ	1
オールラウンダー	1
種子島宇宙センター	1
伝来	1
始興	1
質問主意書	1
人権教育	1
原子炉格納容器	1
門限	1
父と子	1
間体	1
コンプレッサー	1
グリン	1
熱伝導率	1
グラファイト	1
集成材	1
活魚	1
商圏	1
回転率	1
福利厚生	1
いプレゼント	1
職場体験	1
アップライト	1
調律	1
防災センター	1
ウォッチング	1
表札	1
く間	1
私立大	1
国際郵便	1
毒グモ	1
体積	1
高部	1
新山	1
ごちそうさま	1
キャビン	1
サーチライト	1
スチロール	1
救命	1


B-ignoredを考慮したことで、固有名詞を結構取り除けた。

さらに精錬する方法は？

* 形態素解析で分かち書きした境界に一致するか判定

BCCWJでもやってみる。

In [27]:
from entitypedia.evaluation.converter import xml_to_iob2
from seqeval.metrics.sequence_labeling import get_entities
from collections import Counter

bccwj_dir = os.path.join(BASE_DIR, 'bccwj')
X_true, y_true = xml_to_iob2(bccwj_dir, {})
y_pred = [recognizer.iob2(text) for text in X_true]

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
freq_bccwj = Counter(words)
freq100 = sum(v for _, v in freq_bccwj.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq_bccwj.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1056750
Num of types  : 12587
809940 / 1056750
の	67021
に	43255
い	38376
る	35274
て	35149
で	33972
を	33308
と	32678
が	31091
は	29347
な	26453
た	25799
し	20600
っ	19977
す	19754
れ	16759
か	16614
も	15862
ら	14497
ま	13454
こ	12378
う	11962
り	11050
だ	10328
く	10027
あ	9236
した	8878
き	7980
そ	7754
け	7238
さ	7149
ど	6958
ん	6214
め	6098
え	6012
や	5347
わ	4308
つ	4252
お	3825
よ	3542
よう	3233
ば	3047
的	2921
ち	2834
ろ	2692
み	2488
せ	2457
じ	2432
行	2398
人	2214
ず	2108
大	1902
方	1841
中	1716
ね	1701
べ	1699
へ	1650
要	1606
実	1595
しい	1587
び	1521
はな	1443
時	1438
げ	1437
場	1376
とも	1371
より	1363
私	1329
合	1309
対	1303
いの	1302
高	1291
ほ	1275
取	1245
また	1215
一	1213
かな	1212
何	1176
分	1173
せん	1139
しょう	1135
下	1121
む	1104
間	1090
年	1088
本	1071
体	1064
受	1013
ゃ	1003
気	1003
目	1001
手	1000
かし	960
自分	941
通	897
力	890
ご	889
話	882
化	881
生	879
支	864
事	842
向	832
新	818
数	786
自	785
備	775
日	770
面	769
成	764
なかった	759
全	756
定	755
内	752
施	741
初	736
ですか	734
ぐ	730
知	725
関係	722
ぶ	719
変	710
書	688
国	677
無	666
におい	660
地	659
強	657
こう	656
心	643
加	642
部	637
物	634


オフ	14
リハ	14
ゲット	14
センス	14
袖	14
鼻	14
引き出し	14
グラフ	14
地下鉄	14
精度	14
福	14
化粧品	14
性別	14
兆	14
トル	14
遺伝	14
人間性	14
もしかして	14
同級生	14
食材	14
じん	14
りお	14
通訳	14
宴	14
路面電車	14
放出	14
飛ばし	14
さび	14
砂浜	14
温泉	14
府	14
もったいない	14
正規	14
検問	14
清掃	14
床面積	14
賃貸住宅	14
汎用	14
くず	14
革新	14
代替	14
生活環境	14
好奇心	14
富	14
判定	14
視聴者	14
飼料	14
マイ	14
規則	14
砂防	14
韓	14
先進国	14
保護観察	14
教科	14
農産物	14
庁	14
孤立	14
重し	14
情報公開	14
保安林	14
マーケット	14
同期	14
公債	14
団員	14
枝	14
労働時間	14
あふれる	14
電子メール	14
インテリア	14
⌒	14
暮れ	14
のん	14
トラック	14
芽	14
悲劇	14
事務局	14
飢え	14
フィット	14
墓地	14
カーテン	14
ラブ	14
丈	14
予感	14
仕方がない	14
文学	14
絶望	14
霧	14
買収	14
包み	14
農薬	14
ヨット	14
速さ	14
レーザー	14
ムード	14
風船	14
カジュアル	14
世論	14
出馬	14
接種	14
ジャンル	13
すき	13
キャラ	13
ダウンロード	13
クロス	13
踊り	13
メイン	13
姉妹	13
マン	13
警告	13
ボックス	13
エラー	13
感染	13
ブラウザ	13
用途	13
工作	13
つなぐ	13
濡れ	13
財布	13
延期	13
補償	13
モード	13
資本	13
マジ	13
誤解	13
職種	13
解散	13
信念	13
身分	13
切断	13
ライト	13
気象	13
戦力	13
標識	13
洗剤	13
サラ	13
結露	13
鏡	13
呼吸	13
好きな人	13
美術館	13
不倫	13
さだ	13
手帳	13
嘘	13
感心	13
コード	13
妥協	13
寸	13
陶	13
おろし	13
扇形	13
弧	13
レンタカー	13
誰かが	13
ミニ	13


コースター	4
避難指示	4
協賛	4
醍醐	4
車両番号	4
ぺこ	4
そうかな	4
矢印	4
里帰り	4
演歌	4
芯	4
大木	4
峰	4
本性	4
メイク	4
私塾	4
贈り物	4
大分	4
不器用	4
渾身	4
書き下ろし	4
麺	4
命日	4
メイ	4
連想	4
マイル	4
纏	4
冷房	4
ルア	4
プラ板	4
切り落とし	4
本命	4
明後日	4
生命力	4
生活保護	4
トモ	4
できるかな	4
セリ	4
書庫	4
チー	4
ウー	4
撥	4
古代	4
穀物	4
親分	4
心霊	4
ガヤ	4
土俵	4
スリル	4
バンバン	4
ぶっかけ	4
ワクワク	4
ヒンジ	4
激怒	4
お花	4
ふわふわ	4
ピンチ	4
融	4
復調	4
マッチ	4
フィギュア	4
けち	4
ステッカー	4
リアクション	4
相方	4
タコ	4
アカ	4
まなか	4
イフ	4
ニコニコ	4
げす	4
大切な人	4
エント	4
調教	4
成就	4
この人を	4
ばね	4
ピラミッド	4
巻き返し	4
かたわ	4
交易	4
バブル	4
ソフトウェア	4
酒屋	4
精霊	4
海面	4
エー	4
霊魂	4
丘陵	4
クライアント	4
治癒	4
フェティシズム	4
の港	4
おじいさん	4
宝物	4
川底	4
永遠	4
スピリチュアル	4
政治犯	4
細長	4
南西	4
愛する人	4
折り合い	4
元凶	4
命題	4
昔話	4
塵	4
全滅	4
優雅	4
害鳥	4
ほのか	4
貪	4
かまど	4
タイムス	4
寝床	4
蘇	4
カケ	4
木立	4
スナップ	4
ズラ	4
ヨナ	4
大樹	4
スポットライト	4
億	4
靄	4
驚愕	4
サインペン	4
あの日	4
ボール紙	4
丹	4
劣等感	4
優越感	4
月並	4
手押し車	4
ダス	4
栓	4
しんじ	4
はじき	4
同棲	4
亭	4
清算	4
インディ	4
傾き	4
教育問題	4
セリフ	4
実体	4
スタンダード	4
極楽	4
公序良俗	4
省略	4
プレート	4
断熱材	4
末口	4
塗料	4
無垢	4
サイクル	4
らしくない	4
巾着	4
断絶	4
水深	4
ツブ	4
ダイナ	4
木々	4
悲劇的	4
ゲーム機	4
ファッションショー	4
たいよう	4
遥か	4
墓石	4
安全基準	4
くめ	4
現世	4
疎開	4
町

ノブ	2
ラスト	2
消火器	2
借上	2
貸借対照表	2
ディナーショー	2
台形	2
床暖房	2
モルタル	2
酵素	2
クレンジング	2
口コミ	2
標語	2
はやし	2
戟	2
一文	2
チップセット	2
周辺機器	2
クロックアップ	2
リフ	2
光学ドライブ	2
ファースト	2
走行性能	2
サイクリング	2
マフラー	2
サンルーフ	2
乗馬	2
ログハウス	2
クワ	2
バード	2
野生動物	2
カム	2
ピックアップトラック	2
田舎暮らし	2
漁港	2
パラボラアンテナ	2
乾電池	2
高級車	2
モデルチェンジ	2
颯爽	2
華奢	2
セクシー	2
大公	2
ジュエリー	2
ちょっぴ	2
ブラ	2
キュート	2
雪山	2
代名詞	2
南東	2
新天地	2
国歌	2
財界	2
必然性	2
根こそぎ	2
権益	2
教科書採択	2
ミディアム	2
渋み	2
ミリ	2
粘度	2
軍配	2
学区	2
不就学	2
党大会	2
賓	2
右肩	2
内申書	2
指導要録	2
後	2
地域おこし	2
べこ	2
大義名分	2
校歌	2
どん底	2
破壊行為	2
無実	2
風化	2
レスパ	2
国富	2
処方箋	2
リフレーション	2
開かれた社会	2
炎上	2
テロリズム	2
貧富	2
大学通信教育	2
資産デフレ	2
旧友	2
レンタルビデオ	2
ソプラノ	2
黒星	2
緑茶飲料	2
抄	2
画集	2
拒絶反応	2
熱波	2
発破	2
訪問介護	2
診療報酬	2
石油精製	2
模倣	2
シンクタンク	2
妥当性	2
自覚症状	2
外貨準備	2
工作機械	2
ホルモン	2
従属	2
五輪	2
みらい	2
外傷	2
水死	2
ライトアップ	2
パレット	2
地方都市	2
資産凍結	2
茶々	2
テンプレート	2
有利子負債	2
衣替え	2
政治問題	2
公益法人	2
開	2
覇者	2
スタミナ	2
番付	2
連勝	2
却下	2
ざっくばらん	2
胴体	2
ハイジャック	2
氷河	2
半月	2
古墳	2
指名競争入札	2
吹奏楽	2
占い	2
マイナス成長	2
七宝	2
工芸	2
奉	2
起死回生	2
あすか	2
支援団体	2
前哨戦	2
低公害車	2
現地生産	2
創造物	2
沸騰	2
里親	2
ハミング	2
気象台	2
火災保険	2
ストーカー	2
草木	2
ドヤ	

うれしいこと	1
プローブ	1
キャプチャ	1
ランダム	1
ディストリビューション	1
デスクトップ環境	1
コンポーネント	1
オープンソース	1
カレンダー	1
工学部	1
マージン	1
法曹	1
府警	1
スナップ写真	1
ペイン	1
ジミ婚	1
オペレッタ	1
アリア	1
ハンサ	1
ウィンドウ	1
だよん	1
ダンボ	1
ブリ	1
急行	1
発酵食品	1
ナイロン	1
色鉛筆	1
スモー	1
棋譜	1
戦	1
金星	1
本手	1
妙手	1
シボリ	1
ツキアタリ	1
サバキ	1
ケイマ	1
トバ	1
バサ	1
ネダ	1
二子	1
かのこ	1
る会見	1
御用邸	1
御料牧場	1
物納	1
三顧の礼	1
特別番組	1
女帝	1
女系	1
代役	1
伏魔殿	1
宮家	1
大河ドラマ	1
奥さまは	1
コメディドラマ	1
縮図	1
結婚願望	1
入信	1
ネクラ	1
を飼う	1
音痴	1
斜陽	1
凋落	1
一人息子	1
ケロ	1
悲しい人	1
錦	1
まにまに	1
社号	1
長子	1
神像	1
仏具	1
塔頭	1
僧兵	1
梵字	1
難産	1
水瓶	1
ひらり	1
漆	1
回廊	1
岡	1
寄進	1
調合	1
神仏習合	1
名残	1
神饌	1
総社	1
染まるよ	1
滝口	1
マイナスイオン	1
盛り付け	1
懐石	1
谷あい	1
有限会社	1
あやか	1
臼	1
手打ち	1
ネイチャー	1
グラフィティ	1
木道	1
オペラ座	1
小池	1
テーピング	1
入れ	1
体積	1
複合構造	1
スピン	1
アイアン	1
カートリッジ	1
ウェッジ	1
コントラスト	1
アライメント	1
スク	1
螺旋	1
ラケット	1
オールラウンド	1
熱エネルギー	1
プロテクター	1
ニース	1
サスペンション	1
日光	1
砂塵	1
内陸都市	1
町割	1
町家	1
城内	1
居城	1
城郭	1
家老	1
城代	1
要塞	1
チャンバラ	1
不意打ち	1
名城	1
城壁	1
石高	1
鴨居	1
伏流水	1
軟水	1
沿	1
ひとつになって	1
祈願所	1
瓦葺き	1
宝形造	1
御堂	1
廃寺	1
生け簀	1
時間制限	1
モニュメント	1
改宗	1
堕落	1
歴史資料	1
手鏡	1
常勝軍	1
紅白	1
毛皮	1
エッセンス	1
好きっ	1
石油ストーブ	1
涅槃	

毎日新聞とBCCWJの共通部分を洗い出す。

In [28]:
len(set(freq.keys()) & set(freq_bccwj.keys()))

8827

毎日だけにある単語

In [29]:
print(len(set(freq.keys()) - set(freq_bccwj.keys())))
for word in set(freq.keys()) - set(freq_bccwj.keys()):
    print(word)

5373
サーモスタット
正大
茶会
レーザーディスク
学生証
密告
登録名
旋盤
郷土史
泉質
こいのぼり
シャー
友愛
クロスボー
殉職
手引き
沿海地方
生きてゆく
マッチレース
市場競争
オールカラー
赤化
四輪車
埴輪
茶碗
運命のい糸
無農薬栽培
アセスメント
プロ
歴史地震
愛の言葉
通夜
混獲
海難事故
プレゼンス
電源開発
吐き気
支配階級
カフェバー
学生運動
弔慰金
道県
インゴ
デジタルオーディオ
レタックス
政庁
キカ
時がたてば
フレグランス
イージス艦
西高東低
ストーンサークル
上戸
タックスアンサー
裃
ナイーブ
ホロ
司直
頭突き
薪能
ザ・ベスト
パイオニア
末寺
水草
四分
中波
爆破
シンナー
バリトン
ライツ
滑石
モーターショー
宝の山
サッチ
トロフィー
老衰
鍾乳洞
株価暴落
不純物
夜間中学
羽生市
残照
小さな旅館
中川和雄
かつら
水産業
バーテンダー
ペンネーム
デフォルメ
地元新聞
会話教室
金属バット
清新
こども保険
土下座
アクション・ペインティング
経済界
誤認逮捕
笑顔の理由
五部
計量器
変装
冷害
闘士
名誉学位
行命
タホ
大天狗
震源
急場
アラウン
不況
定安
電通
病
カメラマン
蛇行
国民所得
張り子
卒業旅行
タラーリ
新都
クリントン
ブルズ
半球
せん断
京野菜
日生
飛行機雲
ベア
禅譲
人権教育
大時計
天文学
夜空
衛生陶器
中点
強運
クローズ
社会的入院
ドロップ
ごみ収集
金型
ある子供
最多勝利
充電器
続
泥絵
集団検診
哲学者
つり革
マスゲーム
法話
寡占
大曲
腐食
キャラバン
水引
開帳
ガーゼ
銀河
中電
キャッシュカード
ホールセールクラブ
薄化粧
ミナ
青山
ゼット
人工岩
初動態勢
ワット
インベーダー
神経毒
ポーン
劣後ローン
ワイルドカード
蒸気発生器
ワンワン
現場
金網
性差
表と裏
学徒出陣
聞けば聞くほど
食品衛生
箏曲
と世界
粘液
ラブストーリー
政治哲学
戦争犯罪
押し花
地金
パントマイム
飛行禁止空域
幽玄
征服
ママー
大田
ルアー
声帯
チケットぴあ
長打
買い占め
パソコン通信
ビー玉
コミッション
戦記
解釈学
高政
の調
設
高山植物
舞踏会
訴えの利益
受信料
落雷
新城
カッターナイフ
マシー
自助グループ


アンプラグド
悲喜劇
ペナルティ
ワークス
特異点
思想・良心の自由
国民の声
決勝
タオル地
わが町
指圧
液晶ポリマー
ドラフト会議
雑居ビル
四股名
熊手
切腹
パンティー
国営放送
入れ替え戦
分裂選挙
巣立ち
短パン
タペストリー
あくび
北国
登録免許税
英会話
スキーマ
キラー
保育者
ザザ
暗算
服役
カブト
パリジャン
プリズム
ハンドマイク
日研
俳壇
信教の自由
小旗
自治領
歩行器
すだれ
多選
ダンディー
津波注意報
勇者
スパ
アレルギー
大観
新入
塩漬け
画像処理
ノンフィクション
多次元
和気
相似
バリン
泣き虫
みの駅
サンサン
スナック菓子
尊厳死
少女歌劇
ラッカー
放水
吸盤
からくり
課税
脅迫状
融通手形
サイクル安打
領主
制御装置
課税標準
ボイド
絵
ハンドブック
悪役
防犯灯
補数
病原体
抵当権
偵察衛星
トラベラーズチェック
染色体異常
ブリキ
の芸
紙皿
冷戦
一揆
茶飯
名寄市
本人確認
詰み
決壊
住宅展示場
生物化学的要求量
鉱石
弁護士会
改造車
ナウい
聖人
強化ガラス
男役
蔵元
テツ
熱処理
配置図
野良
環礁
モンタージュ
とり天
工事監理
位相
換気扇
重
新進気鋭
表土
伝家の宝刀
農地転用
三々
の警察
海底油田
セクト
基礎化粧品
万国旗
ブラインド
表皮
書き初め
雷管
シーソー
カントリー
神は愛なり
解題
市中金利
在・
ラジオ放送
国民投票
鋼板
チビ
文明の衝突
護衛空母
抗精神病薬
先発ローテーション
中央分離帯
イレブン
平手打ち
ディスポーザー
ぼっち
温泉宿
ハンモック
業法
迷路
模擬店
狂言
鉄道管理局
振り逃げ
アスファルト
元気を出して
突風
未来のために
調律
フリートーク
作人
集団安全保障
洋楽
だま
デイケア
ホリ
大逆転
性愛
国勢調査
映画人
竹馬
連邦政府
コピー用紙
理工学
保釈
社会史
来夏
オール与党
マラン
本土決戦
ば放
転勤族
小江戸
障害者手帳
敵前逃亡
煮物
産業社会
字書
キロ
大新
民主左翼連合
社主
長考
用日
時宜
予備免許
州裁判所
直方体
スポーツ番組
閃光
冗長
電話網
殺処分
自己認識
ジャストミート
高女
ブーメラン
郵便受け
教育行政
川上
日量
三角形
ドル箱
下駅
トランプ
ゼリー
ロッカー
インスパイ

BCCWJだけにある単語

In [30]:
print(len(set(freq_bccwj.keys()) - set(freq.keys())))
for word in set(freq_bccwj.keys()) - set(freq.keys()):
    print(word)

3760
バーミキュライト
篤志面接委員
クラッシュ
シナジー
振り込め詐欺
熱い想い
上着
塩気
オーバーライド
ガチンコ
ペン画
ハム
木戸
名工
框
スペシャルオリンピックス
ネダ
人口分布
別科
悪戯
メータ
ひとつになって
マイナス思考
穴蔵
インディアカ
テ
反面教師
母線
パラ
プロヴァンス料理
千年
バックナンバー
アンソロジー
ホヤ
新規性
コピーバンド
不良少年
いつもそばに
種類株式
土手
外地
外国法人
渚
風に吹かれて
遺族年金
溶岩台地
正視
脱税
在留特別許可
カーソル
フェイ
大元
大浴場
岡
ダコ
サイクリング
コマンド
徴用
フリース
虞
深夜番組
メーリングリスト
人工林
オシ
日伝
館後
鍛錬
て沿
見知らぬ女
警戒宣言
ナノテクノロジー
砲弾
現代語
満天
アカウント
包摂
廃語
実弾
祈る人
イントラネット
入梅
食券
年代測定
加熱調理
新潟
レオ
日本語
計量カップ
衛星電話
ブランディング
パケット
特別区
破産法
興化
虚空
緊急輸送道路
かあちゃん
ライナー
タタ
水蒸気爆発
反応熱
情報収集衛星
朝市
鞭
トライアル
かのこ
ダビング
熟語
地球温暖化
キャブ
御料
三角山
アフィ
タグ
ルチェ
の花
歳暮
予定利率
ビューティ
プロトタイプ
並列
天秤棒
暗殺
保全事業
デイズ
オーバータイム
すあま
潰瘍
通信事業
ポチ
田舎暮らし
メイク
学区
受精
バイオスフィア
マブ
キラキラ☆
の庭
運動器
ストラップ
国民食
すぱすぱ
スピリチュアル
目次
よくあることさ
商品券
ソバ
グリーンベルト
円錐
ケータイ
唯我独尊
踏切
伏流水
ヒートシンク
民間薬
雑種
スプリング
垂直統合
キャパシティ
入れ子
ブラクラ
時の少女
スパム
グチャ
災害補償
コマンドプロンプト
多項式
あなた。
ベイ
簡易保険
移動平均
加入電話
エロイ
路面電車
パケット通信
証券化
啓示
境界条件
ピープル
スメラ
ワクチン
灰皿
バッチ
什器
濁音
レーム
端子
南部地区
ポシェット
凝固
読み聞かせ
フロー
御廃止之議
台地
系統図
限定販売
透水性舗装
毛織物
膝
学士
昇天
ヘドロ
しっこ
防災無線
トラトラトラ
検便
副反応
附則
難升米
チップセット
切り落とし
インターホン
扇形
宇宙旅行
独唱
定員割れ
童貞


ミニカー
鷲
無礼講
有利子負債
一般教書演説
歩道橋
鉤
ローラー
元勲
ダイビング
デジカ
カリカ
キャリア形成
頬
ちまちま
短波放送
つゆ
↓
教育訓練機関
外貨預金
プラットホーム
稲刈り
スーパーコンピュータ
材木問屋
違勅
ハードウェア
ハッキング
納品
南極
ビジネスマン
ゴジ
更生保護
航法
資本ストック
無目
ハイエンド
家船
海面上昇
共済
これからの人生
オナニー
マグ
チ相
フェンダー
拉致
霊柩車
資産凍結
ボーダーライン
パーン
ワード
トレーラーハウス
教育政策
弾道
何も言わずに
ヒック
縁故採用
スペック
放送区域
煙幕
特別永住者
あるじゃん
感想戦
モシ
発破
ジャスト
リラ
木立
エクササイズ
輸送艦
政府広報
泣いているのは
ビュッフェ
狩人
結露
打ちこわし
月齢
リピーター
コチラ
ならず者
肩ツキ
ママゴト
クク
シンプルライフ
偽善
みなと
形容動詞
放漫経営
ビデオライブラリー
石油備蓄
二股
孫会社
油圧ショベル
杞
ウイスキー
日進月歩
借上
鞍
めっけもん
イオ
湊
エンデ
漢字
水鳥
児童会
クリニック
リップサービス
新教育
続柄
の海
中頭
オト
雪対策
交配
無線設備
アドレス
アタッチメント
敷物
ジャズ喫茶
リハビリテーション
展開図
ササ
消耗品
措置制度
計算手
すきま風
方便
基底
携帯灰皿
荒海
まちおこし
ゾーニング
チェックリスト
仙人
零落
耐候性
双子の赤字
ピンポイント
日の後
再生産
オミ
落葉広葉樹林
巨大津波
郵政民営化
ナショナリズム
軟弱地盤
児童虐待
ログ
強くてニューゲーム
シャキ
悲しみの歌


気になる点

* 曖昧さ回避ページとどのくらいかぶるのか？
  * 曖昧さ回避ページを取り除いてから教師データ作るとラベル付する数が減らせるのでよい。